In [3]:
import os
import pandas as pd
import numpy as np
import data_wrangle
import json
from importlib import reload
import preprocess
import sqlite3

import prettytable as pt


#From PCA
from sklearn.decomposition import PCA
from sklearn import decomposition
from sklearn.preprocessing import scale
from sklearn.model_selection import cross_val_score

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import scipy.signal as ss
import pybaselines.spline as py

import pandas as pd
import numpy as np
from math import ceil


# import plotly.io as pio
# pio.renderers.default = "browser"
# pio.renderers.default = "png"



In [3]:
#Detect Oversaturation of spectra
def oversat_check(spectrum, threshold=0.1, window=5):
    """
    Detects oversaturation in a given spectrum by checking if there is a section 
    where the slope of the spectrum is consistently below a given threshold 
    over a given window of points.

    Parameters
    ----------
    spectrum : tuple
        A tuple containing the x and y data of the spectrum to be analyzed.
    threshold : float, optional
        The threshold slope value below which a section of the spectrum is considered 
        to be oversaturated. Default is 0.1.
    window : int, optional
        The size of the window over which to calculate the slopes. Default is 5.

    Returns
    -------
    bool
        True if an oversaturated section is detected, False otherwise.
    """
    x,y = spectrum #Grab x,y data from tuple input
    
    slopes = np.diff(y) / np.diff(x)
    for i in range(len(slopes) - window + 1):
        if np.allclose(slopes[i:i+window], slopes[i], atol=threshold):
            return True
    return False

def unique_count(lst):
    result={}
    for item in lst:
        if item in result:
            result[item] += 1
        else:
            result[item]=1
    return result

In [110]:
path='/home/dunn/DBM/Brain.db'

In [37]:
reload(data_wrangle)
db=data_wrangle.DataTable('/mnt/c/Users/16162/Desktop/Brain.db','sheep')
db.dead_list.append(53)
db.dead_list.append(54)
db.dead_list.append(55)
db.dead_list.append(134) #bad spectra?

db.raman_shifts()
# db.condition='WHERE location IS NOT NULL'
db.condition='WHERE sheep IS NOT 0'

x=db.raman_shifts()
ys=db.intens()

There are 5 found in the dataset.
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
#Reload library to impliment changes
reload(data_wrangle)

#Access dataBase and dataTable
db=data_wrangle.DataTable('/home/dunn/DBM/Brain.db','sheep')

numbs=list(range(1,53))
data=db.get_spectra()
print(data)
data=[(tup[0], tup[1], preprocess.process(tup[2])) for tup in data]
# overs=[oversat_check((d[1], d[2])) for d in data]
# print(overs.count(True))
    

# traces=[go.Scatter(x=d[1], y=d[2], name=d[0]) for d in data]
traces=[go.Scatter(x=d[1], y=d[2], name=d[0]) for d in data]


fig=go.Figure(traces)
# fig.show()

data

[]


[]

<h1> PCA

In [158]:
class multivar:
    def __init__(self,intens,rs,color_by,
                 names, #color_dict,
                 hover=None,symbol=None,
                 ncomp=10):
        self.intens=intens #df
        self.rs=rs #series
        
        self.color=list(color_by)
        self.names=list(names)
        self.ncomp=ncomp

        # self.color_dict=color_dict,

        if hover==None:
            self.hover=self.names
        else:
            self.hover=hover
        if symbol==None:
            self.symbol=self.color
        else:
            self.symbol=symbol
        self.rs_ax='Raman Shift (cm-1)'

    def label_gen(self,prefix,n):
        nums=list(range(1,n+1))
        res=['%s %s'%(prefix, num) for num in nums]
        return res
    
    def symbol_gen(self,len_needed):
        syms=['circle','square','diamond','cross','x','triangle-up','star-triangle-down','triangle-left','triangle-se','pentagon','hexagon2','star','diamond-tall']
        if len_needed <= len(syms):
            return syms[0:len_needed]
        else:
            times_to_mult=ceil((len_needed/len(syms)))
            rep_syms=syms*times_to_mult
            return rep_syms[0:len_needed]

    def pca_run(self):
        pca=decomposition.PCA(n_components=self.ncomp)
        
        #Apply PCA
        X=scale(self.intens)
        X=pca.fit_transform(X)

        #Get components
        pc_labels=self.label_gen('PC',self.ncomp)
        pcs=pca.components_.T*np.sqrt(pca.explained_variance_)
        pcs=pd.DataFrame(pcs,columns=pc_labels, index=self.names)

        #Get explained variance
        explained_list=[round(x*100,2) for x in pca.explained_variance_ratio_]
        explained={pc_labels[i]:explained_list[i] for i in range(self.ncomp)}

        #Get loadings
        loadings=pd.DataFrame(X,columns=pc_labels,index=self.rs)
        return pcs, loadings, explained
    
    def pxfig(self,df,x,y,color,color_dict=None):
        syms=self.symbol_gen(len(df))
        fig=px.scatter(df,x=x,y=y,color=color,
                    #    color_discrete_map=color_dict,
                    #    color_continuous_scale=px.colors.qualitative.T10,
                    #    range_color=(-18,10),
                    #    symbol_sequence=syms, symbol=self.symbol,
                        symbol_sequence=['square','circle','diamond','cross'],
                        symbol=self.color,
                       #symbol_sequence=syms, symbol=self.names, #For color blind friendly figure
                       template='simple_white',hover_name=self.hover)
        
        # fig.update_xaxes(range=[0.8,0.9])
        fig.update_traces(marker_size=10)
        return fig

    def pca_figs(self,x,y,color=None,color_dict=None):
        if color is None:
            color=self.names

        pcs,loadings,explained=self.pca_run()

        #PCA scatter plot
        fig1=self.pxfig(pcs,x,y,color,color_dict=color_dict)
        fig1.update_layout(title='%s vs %s'%(x,y),font_size=20)
        fig1.update_xaxes(title_text='%s (%s%s)'%(x,explained[x],'%'),
                          range=[0.8,1])
        fig1.update_yaxes(title_text='%s (%s%s)'%(y,explained[y],'%'),
                          range=[0,0.5])
        
        # save_name='pc_zoom_mix'
        # pio.write_image(fig,'/mnt/c/Users/16162/Desktop/%s.svg'%save_name,
        #         width=1600,
        #         height=800)
        
        #PCA Loadings
        fig2=px.line(loadings,x=loadings.index,y=x,
                    title='%s (%s%s)'%(x,explained[x],'%'), template='simple_white',
                    labels={x:'','index':self.rs_ax})
        #fig.show()
        fig3=px.line(loadings,x=loadings.index,y=y,
                    title='%s (%s%s)'%(y,explained[y],'%'),template='simple_white',
                    labels={y:'','index':self.rs_ax})
        #fig.show()
        return fig1,fig2,fig3

    def loadings(self,*pcs):
        pcs_df, load, explain = self.pca_run()
        figs=[]

        for pc in pcs:
            title=f'PC {pc}'
            fig=px.line(load, x=self.rs, y=load[title],
                        title='%s (%s%s)'%(title, explain[title], '%'),
                        template='simple_white',
                        labels={title:'', 'x':self.rs_ax})
            figs.append(fig)
        return figs
            
    def pca_3d(self, color=None, color_dict=None, sym=None, x='PC 3', y='PC 2', z='PC 1'):
        if color is None:
            color=self.color
        if sym is None:
            pass
        pcs,loadings,explained=self.pca_run()
        
        fig=px.scatter_3d(pcs,x=x,y=y,z=z, color=color,
                        color_discrete_map=color_dict,
                        symbol=sym,
                        # symbol_sequence=self.symbol_gen(len(color)),
                        # color_continuous_scale=px.colors.sequential.Jet, range_color=(0,-20),  #Jet,
                        hover_name=self.hover
                        )
        fig.update_layout(height=800)#, showlegend=False)#, template='simple_white')
        fig.update_traces()

        fig.update_layout(scene=dict(
            xaxis_title=f'{x} ({explained[x]}%)',
            yaxis_title=f'{y} ({explained[y]}%)',
            zaxis_title=f'{z} ({explained[z]}%)'),
            font=dict(family='Arial', size=18),
            margin=dict(b=50))
        
        # fig.update_layout(scene=dict(xaxis_width=800))
        # fig.update_xaxes(showline=True, linewidth=5, linecolor='black')
        return fig

    def pca_fig(self,xpc,ypc):#,color_dict):
        #delete?
        figs=self.pca_figs(xpc,ypc,color=self.color)
        figs[1].show()
        figs[2].show()

    def pca_fig(self,figs):
        fig=make_subplots(rows=2,cols=2,specs=[[{'rowspan':2},{}],[None,{}]],
                          subplot_titles=[
                          figs[0].layout.title.text,figs[1].layout.title.text,figs[2].layout.title.text],
                          vertical_spacing=0.35)
        for trace in figs[0].data:
            fig.add_trace(trace,row=1,col=1)
            fig.update_xaxes(title_text=figs[0].layout.xaxis.title.text,row=1,col=1)
            fig.update_yaxes(title_text=figs[0].layout.yaxis.title.text,row=1,col=1)
        for trace in figs[1].data:
            fig.add_trace(trace,row=1,col=2)
        for trace in figs[2].data:
            fig.add_trace(trace,row=2,col=2)

        fig.update_xaxes(title_text=figs[1].layout.xaxis.title.text,row=1,col=2)
        fig.update_xaxes(title_text=figs[1].layout.xaxis.title.text,row=2,col=2)
        fig.update_xaxes(title_text=figs[0].layout.xaxis.title.text,row=1,col=1)
        fig.update_yaxes(title_text=figs[0].layout.yaxis.title.text,row=1,col=1)

        fig.update_layout(template='simple_white',
                          font_size=20,
                          font_family='ariel')
        return fig

In [79]:
db.columns()

['id',
 'raman_shift',
 'original_intensity',
 'filename',
 'frame',
 'sheep',
 'date',
 'body_location',
 'surgery',
 'cone',
 'foil',
 'degree',
 'radius',
 'wavelength',
 'power',
 'collect_time',
 'incubation_time',
 'nanotag',
 'spike',
 'spike2']

<h1> Data Work

<h2>Get Data From DB

In [5]:
reload(data_wrangle)
db=data_wrangle.DataTable('/mnt/c/Users/16162/Desktop/Brain.db','sheep')

db.dead_list.append(53)
db.dead_list.append(54)
db.dead_list.append(55)
db.dead_list.append(189)
db.dead_list.append(134) #bad spectra?

db.condition='WHERE sheep in (\'4\') AND radius in (\'25mm\') AND body_location in (\'neck\') AND hide IS NULL AND surgery in (\'post\')'


db.raman_shifts()
x=db.raman_shifts()

ys=db.intens()

There are 2 found in the dataset.
+----------------------------------------------+
|                   Outliers                   |
+----------------------------------------------+
| There are no outliers in the remaining data! |
+----------------------------------------------+
+----------------------------------------------+
There are 2 found in the dataset.
+----------------------------------------------+
|                   Outliers                   |
+----------------------------------------------+
| There are no outliers in the remaining data! |
+----------------------------------------------+
+----------------------------------------------+


In [6]:
namedic=db.names(name_cols=['day'])

lst1=[]
for key in namedic:
    res=namedic[key]
    lst1.append((', ').join(list(res)))

namedic=db.names(name_cols=['sheep'])

sheep=[]
for key in namedic:
    res=namedic[key]
    sheep.append((', ').join(list(res)))

In [7]:
namedic=db.names(name_cols=['nanotag','spike','spike2'])

names=[]
for key in namedic:
    res=namedic[key]
    names.append((', ').join(list(res)))

In [8]:
namedic=db.names(name_cols=['filename'])
hover=[]
for key in namedic:
    res=namedic[key]
    hover.append((', ').join(list(res)))

namedic=db.names(name_cols=['surgery'])
surg=[]
for key in namedic:
    res=namedic[key]
    surg.append((', ').join(list(res)))

namedic=db.names(name_cols=['day','surgery'])
color=[]
for key in namedic:
    res=namedic[key]
    color.append((', ').join(list(res)))

In [ ]:
print(len(x))
print(ys.shape)
print(len(names))

unique_count(color)

In [234]:
colorkey={'no, no, no': 'grey',
 '10min, no, no': '#92dc7e',
 '15min, no, no': '#64c987',
 '20min, no, no': '#39b48e',
 '5min, no, no': '#c4ec74',
 '20min, 0min, no': '#089f8f',
 '30min, 10min, no': '#08737f',
 '35min, 15min, no': '#215d6e',
 '40min, 20min, no': '#2a4858',
 '25min, 5min, no': '#00898a',
 'NBA 0min, no, no': '#fafa6e',
 'both, yes, day2': 'black',
 'both, yes, 0min': '#ff1313',
 'both, yes, 5min': '#ff004a',
 'both, yes, 10min': '#ff007e',
 'both, yes, 15min': '#e500b3',
 'both, yes, 20min': '#a700e5'
 }

<h1> Analysis

In [ ]:
# pio.renderers.default = "svg"
var=multivar(ys.T,x,names,hover,symbol=lst1, hover=hover)#, hover=file)

# figs=var.pca_figs('PC 3', 'PC 3')
# f=figs[1]
# f.update_layout(font=dict(family='Ariel', size=30))
# f.write_image('/mnt/c/Users/16162/Downloads/%s.svg'%'sheep2_pc3', width=1600, height=800)

f=var.pca_3d(color=color, sym=color)#,color_dict=colorkey)
f.update_layout(font=dict(family='Arial', size=18),
                margin=dict(b=50))
# f.write_image('/mnt/c/Users/16162/Downloads/%s.svg'%'sheep2_pca', width=1600, height=1200)
f.show()

In [157]:
var=multivar(ys.T,x,names,hover,symbol=lst1, hover=hover)


<h1>Update Database Find and Replace

In [37]:
reload(data_wrangle)
db=data_wrangle.DataTable('/mnt/c/Users/16162/Desktop/Brain.db','sheep')
# db.condition='WHERE sheep IS 2'
db.data_info('nanotag') #Add 'condition' to find_replace

db.find_replace('2023-04-13', '1','nanotag')

Number of samples selected: 280


[+----------------------+
 |       nanotag        |
 +----------+-----------+
 |  Value   | Frequency |
 +----------+-----------+
 |   0min   |     6     |
 |  10min   |     12    |
 |  15min   |     12    |
 |  20min   |     27    |
 |  25min   |     12    |
 |  30min   |     12    |
 |  35min   |     12    |
 |  40min   |     12    |
 |   5min   |     12    |
 | NBA 0min |     6     |
 |   both   |     87    |
 |    no    |     70    |
 +----------+-----------+]